## Rabi probe
### §PULSE_LENGTH§ns, linearity limit at §LINEARITY_AMP_LIMIT§V, 4 periods at §4_PERIODS§V

In [ ]:
#f.close()
f  = h5py.File(§HDF5_PATH§, 'r', swmr=True)
pa = f['parameters']
da = f['data']
#f.close()

amp   = da['amp'][()]
data1 = da['I1'][()] + 1j * da['Q1'][()]
data2 = da['I2'][()] + 1j * da['Q2'][()] 
data  = IQ_rot((data1 - data2))
amp2  = np.linspace(amp[0], amp[-1], 1000)

#fig, ax = plt.subplots()
#ax.plot(amp, data.real)

# running FFT with {width} points
#fig, ax = plt.subplots()
width = 31
half = int((width-1)/2)
inv_rabi = []
img = []
fmin, fmax = None, None
for center in range(half, amp.size-half):
    sample = data.real[center-half:center+half+1]
    # estimate a_rabi
    rfft = np.abs(np.fft.rfft(sample - np.mean(sample)))
    freq = np.fft.rfftfreq(sample.size, d=(amp[-1]-amp[0])/(amp.size-1))
    inv_rabi.append(freq[np.argmax(rfft)])
    #inv_rabi.append(np.average(freq, weights=rfft**10))
    img.append(rfft)
    fmin, fmax = freq[0], freq[-1]
    #ax.plot(freq, rfft)

ampl = amp[half:-half-1]
ampc = amp[half:-half]
ampr = amp[half+1:-half]
diff = np.abs(np.diff(inv_rabi))
ind  = np.where(diff >= np.amax(diff))
liml = ampl[max(ind)][0]
limr = ampr[max(ind)][0]
amp_lim = (liml+limr)/2
img = np.array(img).T[::-1,::]

#fig, ax = plt.subplots()
#ax.imshow(img, extent=[ampc[0], ampc[-1], fmin, fmax], alpha=0.25, aspect='auto')
#ax.plot(ampc, inv_rabi)
#ax.plot((ampl+ampr)/2, -diff)
#ax.legend((f'1/a_rabi = f(amp)', f'Diff: linearity_amp_limit = {amp_lim:f}'))

# estimate a_rabi
rfft = np.abs(np.fft.rfft(data.real - np.mean(data.real)))
freq = np.fft.rfftfreq(data.real.size, d=(amp[-1]-amp[0])/(amp.size-1))
rabi = 1/freq[np.argmax(rfft)]
print(f'a_rabi ~ {rabi}')

# discard non-linear part
Nstop = np.amin(np.where(amp >= amp_lim))

def cosine(t, a_rabi, b, c):
    t_arr = np.array(t)
    return b*np.cos(2*np.pi*t_arr/a_rabi)+c

(a_rabi, b, c), pcov = opt.curve_fit(cosine, amp[:Nstop], np.real(data[:Nstop]), (rabi, 0, 0))
print(f'a_rabi = {a_rabi}')

result = {
    'linearity_amp_limit': float(f'{amp_lim:f}'),
    '4_periods':           float(f'{4*a_rabi:f}'),
    'samples_per_period':  float(f'{amp[amp <= 4*a_rabi].size/4:f}')
}
globals()['result'] = result
result